In [ ]:
#for using in google colab
from google.colab import drive
drive.mount('/content/drive')
!unzip "drive/MyDrive/train.zip" -d "/content/data/"
!unzip "drive/MyDrive/test.zip" -d "/content/data/"

Streaming output truncated to the last 5000 lines.
 extracting: /content/data/test/images/clip_1b0dd7be5.png  
 extracting: /content/data/test/images/clip_94ef9189e.png  
 extracting: /content/data/test/images/clip_e26a61b40.png  
 extracting: /content/data/test/images/clip_a62241d7c.png  
 extracting: /content/data/test/images/clip_ef44e3828.png  
 extracting: /content/data/test/images/clip_ae4acf582.png  
 extracting: /content/data/test/images/clip_d84feb046.png  
 extracting: /content/data/test/images/clip_9645ce01f.png  
 extracting: /content/data/test/images/clip_09ad13d1f.png  
 extracting: /content/data/test/images/clip_9dba97349.png  
 extracting: /content/data/test/images/clip_080f360a6.png  
 extracting: /content/data/test/images/clip_712ff7ae7.png  
  inflating: /content/data/test/images/clip_0064f7bae.png  
 extracting: /content/data/test/images/clip_1a16fa474.png  
 extracting: /content/data/test/images/clip_8289ef1e5.png  
 extracting: /content/data/test/images/clip_b8c5f

In [ ]:
import os 
import imageio
from IPython.display import display, Image
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# images directory contains spectrograms, mfcc directory contains MFCCs
train_path = 'data/train/images/'
test_path = 'data/test/images/'

#ignore .ipynb_checkpoints
classes = [f for f in os.listdir(train_path) if not f.startswith('.')]
targets = ['yes','no','up','down','left','right','on','off','stop','go','silence','unknown']

#128x87 with 1 channel, expanded for the model
image_shape = (128,87, 3)
num_classes = len(targets)

In [ ]:
def get_image_names():
  images = []
  for c in classes:
    #print(c)
    if c in targets:
      label = targets.index(c)
    elif c == '_background_noise_':
      label = targets.index('silence')
    else: 
      label = targets.index('unknown')
    class_path = train_path + c + '/'
    class_images = os.listdir(class_path)
    labeled = []
    for i in class_images:
      labeled.append([class_path + i,label])
    images.append(labeled)
  return images 

In [ ]:
def split_names(image_list, num_batches=2):
  batches = [[] for _ in range(num_batches)]
  for i in image_list:
    num = len(i)
    random.shuffle(i)
    step = num // num_batches 
    #print(step)
    for k in range(0, num_batches):
      batches[k].append(i[ k * step : min( (k+1)*step, num) ])
  return batches 

In [ ]:
def make_training_batch(image_names):
  train_x, train_y = [], []
  # training images are grouped by class
  for c in image_names:
    for i in c:
      # load the image as a np array
      x = np.array(imageio.imread(i[0]))
      # flip the image so the first axis is timesteps
      x = np.transpose(x)
      # split silence clips into smaller chunks 
      if targets[i[1]] == 'silence':
        # number of samples to take from each noise file 
        num_samples = 100 
        for _ in range(num_samples):
          start = random.randint(0, x.shape[0] - image_shape[0])
          sample = x[start : start + image_shape[0], :]
          sample = np.resize(sample, image_shape)
          train_x.append(sample)
          train_y.append(i[1])
      else:
        # pad to a uniform size for the model 
        x = np.resize(x,image_shape)
        train_x.append(x)
        train_y.append(i[1])
  # convert outer lists to np arrays
  train_x = np.array(train_x)
  train_x = keras.applications.inception_resnet_v2.preprocess_input(train_x)
  train_y = np.array(train_y).reshape(-1,1)
  # transform the labels into one-hot vectors 
  onehot = OneHotEncoder()
  train_y = onehot.fit_transform(train_y).toarray()
  return (train_x, train_y)


In [ ]:
def training_loop(num_times=1, num_batches=1, num_epochs=50, model=None):
  # build the model if needed
  if model is None:
    model = build_model()
  
  # callbacks used in training
  early_stop = EarlyStopping(monitor='val_accuracy',min_delta=.001, patience=5, restore_best_weights=True)
  checkpoint = ModelCheckpoint('best_model.h5',monitor='val_accuracy', save_best_only=True,mode='max')

  # get filenames  of training images, labeled with their classes
  name_list = get_image_names()
  for e in range(num_times):
    print("Loop {}".format(e))
    # divide training data into smaller chunks
    # prevents RAM issues with large models and helps with overfitting
    name_batches = split_names(name_list, num_batches)

    for k in range(num_batches):
      print("Batch {}".format(k))
      train_x, train_y = make_training_batch(name_batches[k])
      shuffler = np.random.permutation(len(train_x))
      train_x = train_x[shuffler]
      train_y = train_y[shuffler]
      model.fit(train_x,train_y,batch_size=50,epochs=num_epochs, callbacks=[early_stop,checkpoint], validation_split = 0.2)

In [ ]:
def build_model():
  model = keras.Sequential()
  model.add(keras.applications.InceptionResNetV2(weights=None,include_top=False, input_shape=(128,87,3)))
  model.add(layers.Flatten())
  model.add(layers.Dense(units=num_classes, activation='softmax'))
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=['accuracy'])
  return model

In [ ]:
model = build_model()
#model.load_weights("best_model.h5")

In [ ]:
training_loop(num_times=1,num_batches=1,num_epochs=100,model=model)

In [ ]:
def read_test_images(num=1000,image_names=[]):
  images = []
  names = []
  for k in range(min(num,len(image_names))):
    if (image_names[k][0] == "."):
      continue
    if (not os.path.exists(test_path + image_names[k])):
      continue 
    names.append(image_names[k])
    x = np.array(imageio.imread(test_path + image_names[k]))
    x = np.resize(x, image_shape)
    #x = np.repeat(x[:, :, np.newaxis], 3, axis=2)
    images.append(x)
  return (names,np.array(images))

In [ ]:
def predict_labels(model, test_x):
  predictions = model.predict(test_x)
  labels = []
  for p in predictions:
    i = np.argmax(p)
    labels.append(targets[i])
  return labels

In [ ]:
def evaluate_model(model):
  test_images = os.listdir(test_path)
  test_num = len(test_images)
  batch_size = 2000
  pred_file = open("predictions.csv","w")
  pred_file.write("fname,label\n")
  for n in range(0,test_num,batch_size):
    image_names, image_batch = read_test_images(batch_size,test_images)
    print(n)
    image_batch = keras.applications.inception_resnet_v2.preprocess_input(image_batch)
    label_batch = predict_labels(model,image_batch)
    for k in range(min(batch_size,len(image_batch))):
      label = label_batch[k]
      im = image_names[k].replace("png","wav")
      pred_file.write("{},{}\n".format(im,label))
    
    test_images = test_images[len(image_batch):]

  pred_file.close()

In [ ]:
evaluate_model(model)